In [ ]:
!pip install pandas openpyxl

In [ ]:
!pip install spacy nltk

In [2]:
import pandas as pd
import numpy as np
import sys, os, ast, re, string, json, pickle
from pathlib import Path
from glob import glob
from collections import defaultdict, Counter

In [3]:
df = pd.read_excel('../data/raw_data.xlsx')

In [4]:
df

,id,name(название),specialization(специализация),responsibilities(Должностные обязанности),requirements(Требования к соискателю),terms(Условия),skills(Ключевые навыки),salary_from,salary_to,object,...,updated_by,position,phone,website,email,image,unique_code,city_code,source_id,link_resource
0,9495846,Арматурщик,Арматурщик,Работа на строительных площадках. Очистных соо...,NaN,NaN,NaN,150000,170000,NaN,...,\N,Арматурщик,NaN,hh,hh@hh.ru,\N,HH-81058502,\N,1,https://hh.ru/vacancy/81058502
1,9495850,Арматурщик,Арматурщик,Гнутье арматурной стали на механических станка...,NaN,NaN,NaN,130000,150000,NaN,...,\N,Арматурщик,NaN,hh,hh@hh.ru,\N,HH-80163818,\N,1,https://hh.ru/vacancy/80163818
2,9495851,Арматурщик,Арматурщик,Вязка арматуры.,NaN,NaN,NaN,85000,90000,NaN,...,\N,Арматурщик,NaN,hh,hh@hh.ru,\N,HH-77729491,\N,1,https://hh.ru/vacancy/77729491
3,2,Арматурщик,Арматурщик,Выполнение работ по гнутью и резке арматурной ...,Выполнение работ по гнутью и резке арматурной ...,NaN,\N,50000,100000,"ООО ""АЛЬМИС_ИНТЕГРАЛ""",...,1,Арматурщик,74959214241,https://almisintegral.ru/,info@almisintegral.ru,\N,\N,\N,1,\N
4,9388857,Арматурщик,Арматурщик Вахта в мcк (питание+проживание) 60/30,Вахта в город Москва. Обязанности: - армирова...,опыт в строительстве приветствуется; работа в ...,продолжительность вахты 60/30 (продление вахты...,NaN,67000,134000,NaN,...,\N,Арматурщик,NaN,avito,avito@avito.ru,\N,A-3037298578,\N,1,https://avito.ru/3037298578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,9306889,Инженер,Инженер конструктор,Обязанности: Проектирование ультразвуковых пре...,NaN,офисные условия,NaN,35000,35000,NaN,...,\N,Инженер,NaN,avito,avito@avito.ru,\N,A-1580492578,\N,1,https://avito.ru/1580492578
995,9306890,Монтажник,Монтажник вентиляции/разнорабочий,"Внимание пока обьявление висит, ведется набор!...",Приветствуется опыт по установке систем кондиц...,Оплата производится раз в неделю по субботам!...,NaN,1800,46800,NaN,...,\N,Монтажник,NaN,avito,avito@avito.ru,\N,A-1578246023,\N,1,https://avito.ru/1578246023
996,9306891,Слесарь-Ремонтник,Слесарь - автомеханик в цех по ремонту карданов,Требуется слесарь в цех по ремонту и балансиро...,Обучаем. Токарные навыки и сварочные навыки пр...,"Зарплата, график работы обсуждаются.",NaN,50000,50000,NaN,...,\N,Слесарь-Ремонтник,NaN,avito,avito@avito.ru,\N,A-928159469,\N,1,https://avito.ru/928159469
997,9306892,Арматурщик,"Оклейщик автомобилей, полиуретан винил, армату...",🔥 К нам в команду требуются: 🔸 Мастер по окле...,"пыт работы, знание и понимание принципов работ...",NaN,NaN,120000,200000,NaN,...,\N,Арматурщик,NaN,avito,avito@avito.ru,\N,A-1047100555,\N,1,https://avito.ru/1047100555


In [5]:
df.columns

Index(['id', 'name(название)', 'specialization(специализация)',
       'responsibilities(Должностные обязанности)',
       'requirements(Требования к соискателю)', 'terms(Условия)',
       'skills(Ключевые навыки)', 'salary_from', 'salary_to', 'object', 'city',
       'address', 'type_of_employment_id(Тип занятости)',
       'expirience(опыт работы)', 'langs(знание языков)', 'rights_id',
       'notes(Примечания)', 'employer_id', 'status_id', 'status_date',
       'dadata_object', 'type_job_id(Вид работы)', 'speciality_id',
       'work_experience_id', 'created_by', 'updated_by', 'position', 'phone',
       'website', 'email', 'image', 'unique_code', 'city_code', 'source_id',
       'link_resource'],
      dtype='object')

In [6]:
import spacy
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/matvey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!pip install --upgrade deepsegment

In [ ]:
!pip install natasha razdel

In [ ]:
from razdel import tokenize, sentenize

In [54]:
REQS_PHRASES = [
    'требуется',
    'требование',
    'требования',
    'обязанности',
    'что мы предлагаем',
    # ''
]

TERMS_PHRASES = [
    'условия',
    # ''
]

In [64]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('russian')

In [65]:
AVAILABLE = string.printable + ''.join([chr(i) for i in range(ord('а'), ord('я') + 1)]) + ''.join([chr(i) for i in range(ord('А'), ord('Я') + 1)]) + 'ёЁ'
punct = ''.join(ch for ch in string.punctuation if ch not in '|')
PUNT_REM = str.maketrans(punct, ' ' * len(punct))


def preprocess_text(text, remove_punct=False, stem=False):
    text = ''.join(ch for ch in text if ch in AVAILABLE)
    text = text.replace('&quot;', ' ')
    text = re.sub(r'\b\d+\b', '|число|', text)

    if remove_punct:
        text = text.translate(PUNT_REM)

    text = re.sub(r'\s+', ' ', text)
    if stem:
        text = ' '.join(stemmer.stem(w) for w in text.split())
    return text.strip()

In [38]:

_FUNNY_REGEX = re.compile(r'(.+):(.+)?')

def extract_thingy_things(text):
    text = preprocess_text(text)
    print(list(tokenize(text)), '\n')

    sents = list(sentenize(text))
    trigger_words = []
    for sent in sents:
        sent = sent.text
        last = _FUNNY_REGEX.findall(sent)
        # if ':' in sent:
        #     print(text, f'{last=}')
        #     break
        trigger_words += last
        # print(f'{sent=} | {last}')

    return trigger_words

In [39]:
TRIGGER_WORDS = Counter()

for _, (resp, reqs, terms, notes) in df[['responsibilities(Должностные обязанности)', 'requirements(Требования к соискателю)', 'terms(Условия)', 'notes(Примечания)']].iterrows():
    things = extract_thingy_things(resp)
    for trigger_word, other in things:
        # print(trigger_word, other)
        TRIGGER_WORDS[trigger_word] += 1

NameError: name 'tokenize' is not defined

In [40]:
common_trigger_words = [tw[0] for tw in TRIGGER_WORDS.most_common(10)]

In [41]:
TRIGGER_WORDS.most_common(10)

[]

In [42]:
common_tokens = Counter()

for _, (resp, reqs, terms, notes) in df[['responsibilities(Должностные обязанности)', 'requirements(Требования к соискателю)', 'terms(Условия)', 'notes(Примечания)']].iterrows():
    text = preprocess_text(resp)
    for tok in tokenize(text):
        common_tokens[tok.text] += 1

NameError: name 'tokenize' is not defined

In [43]:
common_tokens.most_common()

[]

In [44]:
from razdel import sentenize

In [99]:
N = 3

NGRAMS = {
    'resp': Counter(),
    'reqs': Counter(),
    'terms': Counter(),
    'notes': Counter()
}

for _, (resp, reqs, terms, notes) in df[['responsibilities(Должностные обязанности)', 'requirements(Требования к соискателю)', 'terms(Условия)', 'notes(Примечания)']].iterrows():
    for k, text in [('resp', resp), ('reqs', reqs), ('terms', terms), ('notes', notes)]:
        if pd.isna(text):
            continue
        text = str(text)
        if not text.strip():
            continue

        text = preprocess_text(text.lower(), remove_punct=True, stem=True)
        sents = sentenize(text)
        for sent in sents:
            ngrams = nltk.ngrams(sent.text.split(), n=N)
            for ngram in ngrams:
                NGRAMS[k][ngram] += 1

In [111]:
NGRAM_TOP = 100

In [112]:
MC_REQS, _ = zip(*NGRAMS['reqs'].most_common(NGRAM_TOP))
MC_REQS

(('оп', 'работ', 'от'),
 ('от', '|число|', 'год'),
 ('оп', 'работ', 'на'),
 ('работ', 'от', '|число|'),
 ('разряд', '|число|', '|число|'),
 ('оп', 'работ', 'сварщик'),
 ('ручн', 'дугов', 'сварк'),
 ('умен', 'работ', 'ручн'),
 ('работ', 'ручн', 'дугов'),
 ('удостоверен', 'работ', 'сварщик'),
 ('работ', 'сварщик', 'накс'),
 ('сварщик', 'накс', 'будет'),
 ('накс', 'будет', 'преимуществ'),
 ('знан', 'работ', 'на'),
 ('работ', 'на', 'полуавтомат'),
 ('на', 'полуавтомат', 'и'),
 ('полуавтомат', 'и', 'их'),
 ('и', 'их', 'настройк'),
 ('не', 'мен', '|число|'),
 ('желан', 'работа', 'и'),
 ('умен', 'работа', 'с'),
 ('работа', 'и', 'зарабатыва'),
 ('оп', 'работ', 'в'),
 ('без', 'опыт', 'работ'),
 ('|число|', 'х', 'лет'),
 ('с', 'опыт', 'работ'),
 ('оп', 'работ', 'не'),
 ('работа', 'с', 'многокомпонентн'),
 ('с', 'многокомпонентн', 'краск'),
 ('от', '|число|', 'х'),
 ('работ', 'не', 'мен'),
 ('опыт', 'работ', 'всем'),
 ('оп', 'работ', 'по'),
 ('рассматрива', 'без', 'опыт'),
 ('работ', 'сварщик', '

In [113]:
MC_TERMS, _ = zip(*NGRAMS['terms'].most_common(NGRAM_TOP))
MC_TERMS

(('|число|', '|число|', '|число|'),
 ('вахт', '|число|', '|число|'),
 ('раз', 'в', 'месяц'),
 ('|число|', 'раз', 'в'),
 ('работ', '|число|', '|число|'),
 ('график', 'работ', '|число|'),
 ('по', 'тк', 'рф'),
 ('|число|', 'до', '|число|'),
 ('|число|', '|число|', 'руб'),
 ('за', 'счет', 'работодател'),
 ('до', '|число|', '|число|'),
 ('работ', 'вахтов', 'метод'),
 ('за', 'счет', 'компан'),
 ('|число|', '|число|', 'дне'),
 ('вахтов', 'метод', '|число|'),
 ('метод', '|число|', '|число|'),
 ('оформлен', 'по', 'тк'),
 ('с', '|число|', '|число|'),
 ('|число|', '|число|', 'до'),
 ('официальн', 'трудоустройств', 'соц'),
 ('|число|', 'дне', 'и'),
 ('трудоустройств', 'соц', 'пакет'),
 ('дне', 'и', 'тд'),
 ('на', 'карт', 'без'),
 ('карт', 'без', 'задержек'),
 ('до', 'мест', 'работ'),
 ('график', '|число|', '|число|'),
 ('по', '|число|', 'час'),
 ('в', 'месяц', 'на'),
 ('|число|', '|число|', 'по'),
 ('месяц', 'на', 'карт'),
 ('tv', 'wi', 'fi'),
 ('билет', 'до', 'мест'),
 ('и', 'тд', 'по'),
 ('прожи

In [114]:
MC_NOTES, _ = zip(*NGRAMS['notes'].most_common(NGRAM_TOP))
MC_NOTES

(('в', 'наш', 'компан'),
 ('акц', 'привед', 'друг'),
 ('привед', 'друг', 'прем'),
 ('друг', 'прем', '|число|'),
 ('прем', '|число|', 'руб'),
 ('действ', 'акц', 'привед'),
 ('наш', 'компан', 'действ'),
 ('компан', 'действ', 'акц'),
 ('есл', 'вы', 'приезжа'),
 ('вы', 'приезжа', 'с'),
 ('приезжа', 'с', 'друг'),
 ('с', 'друг', 'на'),
 ('и', 'он', 'отрабатыва'),
 ('он', 'отрабатыва', 'не'),
 ('отрабатыва', 'не', 'мен'),
 ('не', 'мен', '|число|'),
 ('|число|', 'руб', 'есл'),
 ('руб', 'есл', 'вы'),
 ('друг', 'на', 'ваканс'),
 ('на', 'ваканс', 'и'),
 ('ваканс', 'и', 'он'),
 ('мен', '|число|', 'смен'),
 ('может', 'сообщ', 'знаком'),
 ('сообщ', 'знаком', 'и'),
 ('знаком', 'и', 'друз'),
 ('и', 'друз', 'о'),
 ('друз', 'о', 'набор'),
 ('о', 'набор', 'специалист'),
 ('набор', 'специалист', 'в'),
 ('специалист', 'в', 'наш'),
 ('|число|', 'смен', 'может'),
 ('смен', 'может', 'сообщ'),
 ('мы', 'ждем', 'соискател'),
 ('ждем', 'соискател', 'на'),
 ('соискател', 'на', 'должност'),
 ('не', 'жди', 'звон'),


In [115]:
sample = df.sample(n=100)

In [116]:
for _, (resp, reqs, terms) in sample[['responsibilities(Должностные обязанности)', 'requirements(Требования к соискателю)', 'terms(Условия)']].iterrows():
    current_reqs = []
    current_terms = []

    for sent in sentenize(resp):
        text = sent.text
        text = preprocess_text(text.lower(), remove_punct=True, stem=True)
        words = text.split()
        ngrams = nltk.ngrams(words, n=N)
        for ngram in ngrams:
            if ngram in MC_REQS:
                current_reqs.append(sent.text)
                break
            elif ngram in MC_TERMS:
                current_terms.append(sent.text)
                break

    print(resp)
    print()
    print('Требования PRED:')
    print('\n'.join(['SENT: ' + s for s in current_reqs]))
    print('Требования TRUE:')
    print(reqs)
    print()
    print('Условия PRED:')
    print('\n'.join(['SENT: ' + s for s in current_terms]))
    print('Условия TRUE:')
    print(terms)
    print('=================')

✔️ РЕКОНСТРУКЦИЯ школ, детских садов, больниц и жилых зданий.   Вахта 184 800р.  РАБОТА В Донецке.  👩‍🚒Объекты под руководством МЧС.  👷👷Строители!!! Корочки НЕ нужны,главное опыт! (Штукатур, маляр)  ЗАРПЛАТА УКАЗАНА ДЛЯ КАЖДОГО СТРОИТЕЛЯ.  📌 3300 РУБЛЕЙ СМЕНА.  📌суточные в конце каждой отработанной недели:  200*7(дней) =1400 рублей.  Вахта 60/30  График работы 7/0  🚃Проезд за счет компании (Покупаем с любого города России)  🏠проживание, 🦺спец. одежда, 🩺Прохождение медосмотра и 🍜3х разовое питание предоставляется БЕСПЛАТНО!  Документы для трудоустройства: паспорт рф, снилс. инн, корочки(если есть) трудовая книжка(если есть)  ❗Оплата между вахтового отдыха от 8 000- 13 000 руб.  💰Акция приведи друга - получи 3000 руб.  💼 Официальное трудоустройство по ТК РФ БЕСПЛАТНО.  ВСЕ ВОПРОСЫ ПО ТЕТЕФОНУ!!!ВРЕМЯ МОСКОВСКОЕ!!!!!

Требования PRED:

Требования TRUE:
Корочки НЕ нужны,главное опыт!

Условия PRED:
SENT: 📌суточные в конце каждой отработанной недели:  200*7(дней) =1400 рублей.
SENT: Вахта 6